In [1]:
import pprint
import requests
import re
import json

import os
import subprocess
import time

In [2]:
home = "/home/rnovikov/workspace/java/performance_rnd/"
duration = 2
hot_iter = 1

In [3]:
class Item:
    build_size = 0
    start_time = 0
    start_heap = 0
    start_heap_gc = 0
    requests_cold = 0
    requests_cold_heap = 0
    requests_hot = 0
    requests_hot_heap = 0
    
def heap_usage(pid):
    cmd = "jstat -gc " + str(pid) + " | tail -n 1 | awk '{split($0,a,\" \"); sum=a[3]+a[4]+a[6]+a[8]; print sum}'"
    return float(subprocess.check_output(cmd, shell=True).decode('utf-8').strip())
def full_gc(pid):
    out = subprocess.check_output(['jcmd', str(pid), 'GC.run']).decode('utf-8').strip()
    if "successfully" in out:
        print("Full GC")
    else:
        print("Problem with full gc")
def request_per_seconds(url):
    p = subprocess.Popen(['wrk','-t8','-c1000','-d{:d}s'.format(duration),'http://127.0.0.1:8080/'+url], stdout=subprocess.PIPE)
    for line in p.stdout.readlines():
        utf8line = line.decode('utf-8').strip()
        if "Requests/sec" in utf8line:
            print(utf8line)
            return float(re.findall("\d+\.\d+", utf8line)[0])    
def build(url):
    os.chdir(home + url)
    start = time.time()
    subprocess.check_output(["gradle", "clean", "build"])
    evaluate = time.time() - start
    print("Build created in {:f} s".format(evaluate))
    return url
    
def print_item(title, item):  
    print(title)
    print("Build size: {:.2f}mb".format(item.build_size))
    print("Start in {:.2f}s, heap: [{:.2f}/{:.2f}]mb"
          .format(item.start_time, item.start_heap, item.start_heap_gc))
    print("Cold: {:10.2f}s {:7.2f}mb".format(item.requests_cold,item.requests_cold_heap))
    print("Hot:  {:10.2f}s {:7.2f}mb".format(item.requests_hot,item.requests_hot_heap))
    
def test_all(projects):
    results = {}
    for project in projects:
        results[project] = run_test(build(project), "test")
    for key, value in results.items():
        print_item("\n" + key.upper(), value)  
        
def run_test(project, url):
    item = Item()
    item.build_size = os.path.getsize(home + project +'/build/libs/fatjar.jar') / 1000000
    start = time.time()
    p = subprocess.Popen(['java', '-jar', 'build/libs/fatjar.jar'], shell=False, stdout=subprocess.PIPE)
    for line in p.stdout:
        utf8line = line.decode('utf-8')
        if "Started " in utf8line:
            item.start_time = time.time() - start
            print(project.upper() + " was started")
            item.start_heap = heap_usage(p.pid) / 1000
            full_gc(p.pid)
            item.start_heap_gc = heap_usage(p.pid) / 1000
            
            # test
            item.requests_cold = request_per_seconds(url)
            item.requests_cold_heap = heap_usage(p.pid) / 1000
            [request_per_seconds(url) for _ in range(hot_iter)]
            item.requests_hot = request_per_seconds(url)
            item.requests_hot_heap = heap_usage(p.pid) / 1000
            
            p.terminate()
            break
    p.wait() 
    return item

In [9]:
projects = ["springboot_rest", "springboot_rest_db", 
            "vertx_rest", "vertx_rest_jdbc", "vertx_rest_db"]
projects = ["springboot_thymeleaf"]
test_all(projects)

Build created in 3.977154 s
SPRINGBOOT_THYMELEAF was started
Full GC
Requests/sec:   5142.10
Requests/sec:  11249.37
Requests/sec:  23850.99

SPRINGBOOT_THYMELEAF
Build size: 21.23mb
Start in 1.99s, heap: [89.05/18.28]mb
Cold:    5142.10s  177.05mb
Hot:    23850.99s  303.73mb


In [33]:
spring_rest = spring(build("springboot_rest"), "test")
vertx_rest = vertx(build("vertx_rest"), "test")

print_item("\nSPRING REST", spring_rest)
print_item("\nVERTX REST", vertx_rest)

Build created in 2.768953 s
Spring was started
Full GC
Requests/sec:   4737.10
Requests/sec:   9346.39
Requests/sec:  14449.77
Build created in 3.897551 s
Vertx was started
Full GC
Requests/sec:  50976.18
Requests/sec: 206606.21
Requests/sec: 310253.01

SPRING REST
Build size: 14.41mb
Start in 1.78s, heap: [98.56/14.87]mb
Cold:    4737.10s  229.34mb
Hot:    14449.77s  276.81mb

VERTX REST
Build size: 6.30mb
Start in 0.36s, heap: [40.14/7.87]mb
Cold:   50976.18s   28.61mb
Hot:   310253.01s   97.11mb


In [227]:
print(subprocess.check_output(["gradle", "-v"]).decode('utf-8'))


------------------------------------------------------------
Gradle 3.5
------------------------------------------------------------

Build time:   2017-04-10 13:37:25 UTC
Revision:     b762622a185d59ce0cfc9cbc6ab5dd22469e18a6

Groovy:       2.4.10
Ant:          Apache Ant(TM) version 1.9.6 compiled on June 29 2015
JVM:          1.8.0_131 (Oracle Corporation 25.131-b11)
OS:           Linux 4.8.0-58-generic amd64




In [27]:
vertx = vertx(build("vertx_rest"), "test")

print_item("\nVERTX", vertx)

Build created in 3.877910 s
Vertx was started
Full GC
Requests/sec: 217435.66
Requests/sec: 303072.51
Requests/sec: 300485.08

VERTX
Build size: 6.30mb
Start in 0.36s, heap: [40.14/7.87]mb
Cold:  217435.66s   54.77mb
Hot:   300485.08s  150.21mb


In [127]:
request_per_seconds("test")

47732.38

In [172]:
os.path.getsize(home + 'springboot/build/libs/springboot-0.1.0.jar')

14409344